In [163]:
from collections import Counter
from math import log2, ceil
import numpy as np
from itertools import combinations
import heapq
set_print = True


### Class truth_table:
1. truth table for a function, takes len(variable_list), variable_list and output_list as inputs
2. Can fetch sub_table wrt any variable and any value
3. Can evaluate function for any set of inputs
4. First variable in list of variables_list is LSB of binary form of inputs

In [ ]:
class truth_table:
    def __init__(self, n, variables_list, output_list):
        self.n = n                                                                  # number of variables
        self.output_list = output_list                                              # output list, size = 2**n
        self.variables_list = variables_list                                        # list of variables, size = n

    def fetch_output(self, inputs):
        return self.output_list[int(''.join(map(str,inputs)), 2)]                   # Fetch output from output list using binary representation of inputs
    
    def fetch_sub_table(self, x_i, value):
        idx = self.variables_list.index(x_i)

        list_vars = self.variables_list[:idx] + self.variables_list[idx+1:]         # Remove x_i from list of variables

        idx = self.n - idx - 1

        sub_table = []
        for i in range(2**len(list_vars)):
            inputs = [int(x) for x in format(i, '0' + str(len(list_vars)) + 'b')]   # Generate binary representation of inputs
            inputs.insert(idx, value)                                               # Insert value of x_i at idx
            sub_table.append(self.fetch_output(inputs))                             # Fetch output from output list using inputs                    
        return truth_table(self.n - 1, list_vars, sub_table)                        # Return sub-table
    
    def print_table(self):
        print("Truth table:")
        print("Variables:", self.variables_list)
        for i in range(2**self.n):
            inputs = [int(x) for x in format(i, '0' + str(self.n) + 'b')]           # Generate binary representation of inputs
            print(''.join(map(str,inputs)), self.fetch_output(inputs))              # Print inputs and output
        print("------------------------")

In [165]:
variables_list = ['x1', 'x2', 'x3'] 
output_list = [1, 0, 1, 0, 0, 1, 0, 1]  # Output list for 3 variables
n = len(variables_list)

table = truth_table(n, variables_list, output_list)

print("Output for inputs [1, 0, 1]:", table.fetch_output([1, 0, 1]))
print("------------------------")

sub_table = table.fetch_sub_table('x3', 1)
print("Sub-table for x1 = 1:")
sub_table.print_table()                                                             # Here LSB of binary representation is x1 and MSB is x2 in the table

Output for inputs [1, 0, 1]: 1
------------------------
Sub-table for x1 = 1:
Truth table:
Variables: ['x1', 'x2']
00 0
01 1
10 0
11 1
------------------------


### Class RM:
1. Converts input truth_table to RM and also List_of_Lists form for RM
2. Evaluates output for any set of inputs

In [166]:
class RM:
    def __init__(self, truth_table):
        self.truth_table = truth_table
        self.variables_list = truth_table.variables_list                                        # List of variables
        self.n = truth_table.n                                                                  # Number of variables                           
        self.output_list = truth_table.output_list                                              # Output list     
        self.Reed_Muller = self.compute_RM(self.truth_table)                                    # Reed-Muller form
        self.RM_listOfLists = self.RM_to_list_of_lists()                                        # Reed-Muller form as list of lists             

    def compute_RM(self, truth_table):
        if truth_table.n == 1:
            if truth_table.output_list[0] == 0 and truth_table.output_list[1] == 0:             # f(0) = f(1) = 0
                return ['0']
            elif truth_table.output_list[0] == 0 and truth_table.output_list[1] == 1:           # f(0) = 0, f(1) = 1
                return truth_table.variables_list
            elif truth_table.output_list[0] == 1 and truth_table.output_list[1] == 0:           # f(0) = 1, f(1) = 0
                return ['1' , truth_table.variables_list[0]]
            else:                                                                               # f(0) = f(1) = 1
                return ['1']
            
        fx = truth_table.fetch_sub_table(truth_table.variables_list[0], 1)                      # fx  = f(x, x_n = 1)
        f_x_bar = truth_table.fetch_sub_table(truth_table.variables_list[0], 0)                 # f_x_bar = f(x, x_n = 0)

        RM_fx = self.compute_RM(fx)                                                             # RM(fx)                                 
        RM_f_x_bar = self.compute_RM(f_x_bar)                                                   # RM(f_x_bar)              

        x_fx = []                                                                               # x * RM(fx)                       
        for x in RM_fx:
            if x == '0':                                                                        # Remove 0 from RM          
                continue
            elif x == '1':                                                                      # x * 1 = x
                x_fx.append(truth_table.variables_list[0])
            else:                                                                               # x * x_i = x * x_i                            
                x_fx.append(truth_table.variables_list[0] + x)

        x_fx_bar = []                                                                           # x * RM(f_x_bar)      
        for x in RM_f_x_bar:
            if x == '0':                                                                        # Remove 0 from RM          
                continue
            elif x == '1':                                                                      # x * 1 = x
                x_fx_bar.append(truth_table.variables_list[0])  
            else:                                                                               # x * x_i = x * x_i
                x_fx_bar.append(truth_table.variables_list[0] + x)  

        RM = x_fx + x_fx_bar + RM_f_x_bar                                                       # RM = x * RM(fx) + x * RM(f_x_bar) + RM(f_x_bar)

        if '0' in RM:                                                                           # Remove 0 from RM                                     
            RM.remove('0')

        RM = [key for key, value in Counter(RM).items() if value%2 == 1]                        # Remove duplicates from RM

        return RM
    
    def RM_to_list_of_lists(self):                                                              # Converts list of Reed-Muller terms to list of lists
        RM = self.Reed_Muller
        RM_list = []
        for term in RM:
            term_list = []
            for var in self.truth_table.variables_list:
                if var in term:
                    term_list.append(var)
            if term_list == []:
                term_list.append('1')
            RM_list.append(term_list)

        return RM_list
    
    def print_RM(self):
        # print("Reed-Muller form:")
        # RM = self.Reed_Muller
        # for term in RM:
        #     print(term)
        # print("------------------------")
        #print("Variables:", self.variables_list)
        
        print("Reed-Muller form as list of lists:")
        for term in self.RM_listOfLists:
            print(term)
        print("------------------------")

    def evaluate_RM(self, inputs):                                                              # Evaluate Reed-Muller form for given inputs
        RM = self.RM_listOfLists
        output = 0
        for term in RM:
            value = 1                                                                       
            for var in term:
                if var == '1':                                                                  # If term is 1, value = value * 1                   
                    value *= 1
                else:
                    idx = self.variables_list.index(var)                                
                    value *= inputs[idx]                                                        # If term is x_i, value = value * val(x_i)                
            output += value                                                                     # Add value to output                            
        return output % 2                                                                       # Return output mod 2                       
    
    def update_function_check(self, new_literals, expression, alter_literal):                   # Update function with new literal and return Reed-Muller form as list of lists
        new_RM = []
        temp_list = [term for term in new_literals]                                      # Copy of Reed-Muller form
        for term in temp_list:
            new_term = term.copy()                                                              # Copy of term                    
            if alter_literal not in term:                                                       # If term does not contain alter_literal, add term as it is to new_RM        
                new_RM.append(new_term)
                continue
            else:
                temp_term = [literal for literal in new_term if literal != alter_literal]       # Remove alter_literal from term
                for element in expression:                                                      # Add new terms to new_RM                         
                    new_temp_term = temp_term.copy()
                    for i in range(len(element)):
                        if element[i] != '1':
                            new_temp_term = new_temp_term + [element[i]]
                        if new_temp_term == []:
                            new_temp_term = ['1']
                    new_RM.append(new_temp_term)
        print("Before Sorting Loop:")
        print(new_RM)
        print("------------------------")

        for i in range(len(new_RM)):
            new_RM[i] = [key for key, value in Counter(new_RM[i]).items()]
            new_RM[i].sort()                                                                    # Sort individual terms in new_RM so that checking for repeated terms is easier
        print("After Sorting Loop:")
        print(new_RM)
        print("------------------------")

        # Check for repeated terms
        for term in new_RM:
            if new_RM.count(term)%2 == 0:
                # Remove all occurences of term
                while term in new_RM:
                    new_RM.remove(term)
            else:
                # Remove all but one occurence of term
                while new_RM.count(term) > 1:
                    new_RM.remove(term)
        print("After Repeated Terms:")
        print(new_RM)
        print("------------------------")
        final_RM = []                                                                           # Just to replace variable names, say ['x1', 'x2'] with ['a', 'b'] 
        for term in new_RM:
            new_term = term.copy()
            for i in range(len(new_term)):
                if new_term[i] in self.variables_list:
                    new_term[i] = new_literals[self.variables_list.index(new_term[i])]
            final_RM.append(new_term)
        print("After Final RM:")
        print(final_RM)
        print("------------------------")
        return final_RM

    def update_function(self, new_literals, expression, alter_literal):                          # Update function with new literal and update Reed-Muller form as list of lists
        self.RM_listOfLists = self.update_function_check(new_literals, expression, alter_literal)
        self.variables_list = new_literals
        self.print_RM()
        

In [167]:
RM_form = RM(table)
RM_form.print_RM()

inputs = [1, 1, 0]
print("Output for inputs [1, 1, 0]:", RM_form.evaluate_RM(inputs))

print("------------------------")

Reed-Muller form as list of lists:
['x1']
['1']
['x3']
------------------------
Output for inputs [1, 1, 0]: 0
------------------------


In [168]:
table2 = truth_table(2, ['x1', 'x2'], [0, 1, 1, 1])
table2.print_table()

RM_form2 = RM(table2)
RM_form2.print_RM()

Truth table:
Variables: ['x1', 'x2']
00 0
01 1
10 1
11 1
------------------------
Reed-Muller form as list of lists:
['x1']
['x1', 'x2']
['x2']
------------------------


In [169]:
class Solver:
    def __init__(self, size, literals, list_truth_tables):
        self.size = size
        self.literals = literals
        self.list_truth_tables = list_truth_tables
        self.truth_tables = []
        for table in list_truth_tables:
            self.truth_tables.append(truth_table(self.size, self.literals, table))
        self.print_vars()

        self.list_truth_tables, self.size = self.update_truth_tables()  # Update truth tables to remove repeated terms and extend bits
        self.truth_tables = []
        self.literals = []                                              # Update literals list                   
        for i in range(self.size):
            self.literals.append('x' + str(i+1))

        for table in self.list_truth_tables:
            self.truth_tables.append(truth_table(self.size, self.literals, table))
        self.print_vars()

        # Convert list of lists to Reed-Muller form
        self.RMs = []
        for table in self.truth_tables:
            self.RMs.append(RM(table))
            self.RMs[-1].print_RM()
            print("------------------------")

    def print_vars(self):
        print("------------------------")
        print("Size of function:", self.size)
        print("------------------------")
        print("Variables:", self.literals)
        print("------------------------")
        print("Truth tables:")
        for table in self.truth_tables:
            table.print_table()
        print("------------------------")

    def update_truth_tables(self):
        terms = []
        print("Update Truth Tables:")
        for i in range(len(self.list_truth_tables[0])):
            num = 0
            for j in range(len(self.list_truth_tables)):
                num = num + self.list_truth_tables[j][i] * (2**j)
            terms.append(num)
        print("Original Terms:")
        print(terms)
        print("------------------------")

        new_terms = []
        # Use counter to find repeated terms
        counts = Counter(terms)
        print("Counts:")
        print(counts)
        print("------------------------")

        # Print repeated terms
        repeated = []
        for key, value in counts.items():
            if value > 1:
                repeated.append(tuple((key, value)))
        print("Repeated Terms:")
        print(repeated)
        print("------------------------")

        # Bit extension = log2(max_repeated_term)
        if repeated == []:
            bit_extension = 0
            print("Max Repeated Term: None")
            print("Max Repetiton: None")
            print("Bit Extension: 0")
            print("------------------------")
            return self.list_truth_tables, self.size
        max_repeat = max(repeated, key = lambda x: x[1])
        bit_extension = ceil(log2(max_repeat[1]))
        print("Max Repeated Term:", max_repeat[0])
        print("Max Repetiton:", max_repeat[1])
        print("Bit Extension:", bit_extension)

        # Update terms
        for i in range(len(terms)):
            new_terms.append(terms[i] * (2**bit_extension) + counts[terms[i]] - 1)
            counts[terms[i]] = counts[terms[i]] - 1

        self.size = self.size + bit_extension
        for i in range(2**self.size):
            if i not in new_terms:
                new_terms.append(i)
        print("New Terms:")
        print(new_terms)
        print("------------------------")

        # # Use counter to find repeated terms
        # counts = Counter(new_terms)
        # print("Counts:")
        # print(counts)
        # # Print repeated terms
        # repeated = []
        # for key, value in counts.items():
        #     if value > 1:
        #         repeated.append(tuple((key, value)))
        # print("Repeated Terms:")
        # print(repeated)
        # print("------------------------")
        # print("------------------------")

        # Now convert new_terms to list_truth_tables
        new_list_truth_tables = []
        for i in range(self.size):
            new_list_truth_table = []
            for j in range(len(new_terms)):
                new_list_truth_table.append((new_terms[j] >> i) & 1)
            new_list_truth_tables.append(new_list_truth_table)

        print("Original List Truth Tables:")
        for table in self.list_truth_tables:
            print(table)
        print("------------------------")
        print("New List Truth Tables:")
        for table in new_list_truth_tables:
            print(table)
        print("------------------------")
        return new_list_truth_tables, self.size
    


In [170]:
list_truth_tables = [[1, 1, 1, 1], [0, 0, 0, 0]]
literals = ['x1', 'x2']
size = 2

solver = Solver(size, literals, list_truth_tables)


------------------------
Size of function: 2
------------------------
Variables: ['x1', 'x2']
------------------------
Truth tables:
Truth table:
Variables: ['x1', 'x2']
00 1
01 1
10 1
11 1
------------------------
Truth table:
Variables: ['x1', 'x2']
00 0
01 0
10 0
11 0
------------------------
------------------------
Update Truth Tables:
Original Terms:
[1, 1, 1, 1]
------------------------
Counts:
Counter({1: 4})
------------------------
Repeated Terms:
[(1, 4)]
------------------------
Max Repeated Term: 1
Max Repetiton: 4
Bit Extension: 2
New Terms:
[7, 6, 5, 4, 0, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15]
------------------------
Original List Truth Tables:
[1, 1, 1, 1]
[0, 0, 0, 0]
------------------------
New List Truth Tables:
[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
------------------------
------------------------
Size of fu

In [ ]:
class IterativeSolver:
	def __init__(self, rm_list, literals, params):
		self.rm_list = rm_list
		self.literals = literals
		self.size = len(rm_list)
		
		self.iterator = 0
		self.pq = []
		self.ans = []
		self.best_depth = 0

		self.alpha = params[0]
		self.beta = params[1]
		self.gamma = params[2]

		depth = 1
		elim = 0
		term_count = 0
		ans = []
		for rm in rm_list:
			term_count += len(rm)
		self.best_term_count = term_count

		# depth = 0, elim = 0, size = term_count 
		push_element = [rm_list, depth, elim, term_count, ans]
		heapq.heappush(self.pq, (self.alpha*(depth) + self.beta*(elim/depth) - self.gamma*(term_count), push_element))
		self.Gen_op_RM_List()
		self.Recursive_algo()
		
	def Gen_op_RM_List(self):
		self.options = []
		for i in range(self.size):
			literal = self.literals[i]
			other_literals = [x for x in self.literals if x != literal]
			self.options.append([[[literal], ['1']], literal])
			result = []
			n = len(other_literals)
			for r in range(1, n + 1):  # r is the size of each combination
				result.extend(combinations(other_literals, r))
			for r in result:
				self.options.append([[[literal], r], literal])
		return self.options
	
	def options_for_rm_list(self):
		return self.options

	def rm_list_obtained_after_checking(self, RM_list, option):
		final_rm_list = []
		for i in range(self.size):
			final_rm_list.append(RM_form.update_function_check(RM_list[i], option[0], option[1]))
		return final_rm_list
	
	def Recursive_algo(self):
		pop_element = heapq.heappop(self.pq)[1]
		rm_list = pop_element[0]
		og_depth = pop_element[1]
		og_elim = pop_element[2]
		og_term_count = pop_element[3]
		og_ans = pop_element[4]

		space = "--" * og_depth
		if (set_print): 
			print (space, "Current depth is : ", og_depth)
		if (set_print): 
			print (space, "Current functions are : ")
		if (set_print): 
			for i in range(self.size):
				print (space, rm_list[i])
		space += "--"
		
		if (og_term_count == self.size):
			print("SOLVED")
			self.ans = og_ans
		else:
			#options = self.options_for_rm_list(rm_list)
			options = self.options_for_rm_list()
			print("Options are: ", options)
			for option in options:
				new_rm_list = self.rm_list_obtained_after_checking(rm_list, option)
				if (set_print): 
					print (space, "substituting : ", option)
				if (set_print): 
					print (space, "New functions are : ")
				if (set_print): 
					for i in range(self.size):
						print (space, new_rm_list[i])
				new_depth = og_depth + 1
				new_elim = 0
				new_term_count = 0
				new_ans = og_ans + [option]
				for rm in new_rm_list:
					new_term_count += len(rm)
				
				if (new_term_count < self.best_term_count):
					self.best_term_count = new_term_count
				self.iterator += 1
				print (self.iterator, self.best_term_count, new_term_count)
				
				new_elim = og_term_count - new_term_count
				push_element = [new_rm_list, new_depth, new_elim, new_term_count, new_ans]
				if (new_elim >= 0 and new_term_count<=self.best_term_count):
					print("Pushing element with depth", new_depth, "best_count", self.best_term_count, "term_count", new_term_count, "RM_list", new_rm_list)	
					heapq.heappush(self.pq, (self.alpha*(new_depth) + self.beta*(new_elim/new_depth) - self.gamma*(new_term_count), push_element))
			self.Recursive_algo()

In [172]:
# defining files that are being printed into
#table_fixer = open("table_fixer.txt", "w")
#debug_messages = open("debug_messages.txt", "w")
general_output = open("general_output.txt", "w")
set_print = True

# defining problem input
list_truth_tables = [[1, 1, 1, 1, 0, 0, 0, 0],
			   [0, 1, 1, 0, 0, 0, 1, 1],
               [0, 0, 1, 1, 1, 1, 0, 1]]

# making the function reversible 
solver_instance = Solver(3, ['x1', 'x2', 'x3'], list_truth_tables)
table, size = solver_instance.update_truth_tables()
print(table, size)
literals = []
for i in range(size):
	literals.append(chr(ord('a') + i))

# getting all the RMs
print ("--------------------------------", file = general_output)
RM_list = []
for i in range(size):
	truthtable = truth_table(size, literals, table[i])
	ReedMuller = RM(truthtable)
	RM_list.append(ReedMuller.RM_listOfLists)
	print(f"Function {i+1}: {ReedMuller.RM_listOfLists}", file = general_output)
print ("--------------------------------", file = general_output)

# solving the problem
parameters = [0, 0, -1]
solver = IterativeSolver(RM_list, literals, parameters)
for step in solver.ans:
	print(step, file = general_output)

general_output.close()

------------------------
Size of function: 3
------------------------
Variables: ['x1', 'x2', 'x3']
------------------------
Truth tables:
Truth table:
Variables: ['x1', 'x2', 'x3']
000 1
001 1
010 1
011 1
100 0
101 0
110 0
111 0
------------------------
Truth table:
Variables: ['x1', 'x2', 'x3']
000 0
001 1
010 1
011 0
100 0
101 0
110 1
111 1
------------------------
Truth table:
Variables: ['x1', 'x2', 'x3']
000 0
001 0
010 1
011 1
100 1
101 1
110 0
111 1
------------------------
------------------------
Update Truth Tables:
Original Terms:
[1, 3, 7, 5, 4, 4, 2, 6]
------------------------
Counts:
Counter({4: 2, 1: 1, 3: 1, 7: 1, 5: 1, 2: 1, 6: 1})
------------------------
Repeated Terms:
[(4, 2)]
------------------------
Max Repeated Term: 4
Max Repetiton: 2
Bit Extension: 1
New Terms:
[2, 6, 14, 10, 9, 8, 4, 12, 0, 1, 3, 5, 7, 11, 13, 15]
------------------------
Original List Truth Tables:
[1, 1, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 0, 1]
-----------------